In [1]:
import pandas as pd
import joblib
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# =========================
# LOAD DATA
# =========================
data = pd.read_csv("dataset.csv")

X = data.iloc[:, :-1].astype(float).values
y = data.iloc[:, -1].values

print("Dataset shape:", X.shape)
print("Classes:", set(y))

Dataset shape: (5200, 63)
Classes: {'V', 'W', 'F', 'R', 'L', 'Q', 'C', 'Y', 'H', 'E', 'G', 'Z', 'O', 'U', 'X', 'A', 'P', 'K', 'S', 'B', 'D', 'M', 'T', 'I', 'N', 'J'}


In [3]:
# =========================
# ENCODE LABELS (A–Z → 0–25)
# =========================
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# =========================
# SCALE FEATURES
# =========================
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# =========================
# TRAIN TEST SPLIT
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape)
print("Test:", X_test.shape)


Train: (4160, 63)
Test: (1040, 63)


In [5]:
# =========================
num_classes = len(np.unique(y))

In [6]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.4),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(64, activation='relu'),

    Dense(num_classes, activation='softmax')
])

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,226 (231.35 KB)

 Trainable params: 59,226 (231.35 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# =========================
# TRAIN
# =========================
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)



In [9]:
history = model.fit(
    X_train, y_train,
    epochs=120,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]
)

Epoch 1/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5347 - loss: 1.7620 - val_accuracy: 0.9971 - val_loss: 0.0359
Epoch 2/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9854 - loss: 0.0788 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 3/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9924 - loss: 0.0381 - val_accuracy: 1.0000 - val_loss: 7.0225e-04
Epoch 4/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9964 - loss: 0.0171 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 5/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9941 - loss: 0.0168 - val_accuracy: 0.9990 - val_loss: 0.0035
Epoch 6/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9946 - loss: 0.0156 - val_accuracy: 1.0000 - val_loss: 6.4405e-05
Epoch 7/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9998 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 5.4705e-05
Epoch 8/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9960 - loss: 0.012

In [10]:
loss, acc = model.evaluate(X_test, y_test)
print("Final Accuracy:", acc)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 1.7594e-08
Final Accuracy: 1.0


In [11]:
# SAVE MODEL + PREPROCESSING
# =========================
model.save("asl_sign_model.h5")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(encoder, "label_encoder.pkl")

print("Model saved successfully ✅")

Model saved successfully ✅
